# Optimising Parameters of the mini CNN

## Imports

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, roc_curve, roc_auc_score,f1_score

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D,MaxPool2D, InputLayer, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import backend as K
import keras
import tensorflow as tf

import pickle

## Load data and split data

In [ ]:
output_prefix = "03MLouput20240811v001.keras"

with open('all_data.pkl', 'rb') as f:
    images, labels = pickle.load(f)

labels = to_categorical(labels)

# Step 3: Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.4, random_state=42)

## Custom F1 score metric

In [2]:
#@keras.saving.register_keras_serializable()
def f1_metric(y_true, y_pred):
    y_true = K.cast(y_true, 'int32')
    y_pred = K.cast(K.round(y_pred), 'int32')
    TP = K.sum(K.cast(y_true * y_pred, 'float'), axis=0)
    TN = K.sum(K.cast((1 - y_true) * (1 - y_pred), 'float'), axis=0)
    FP = K.sum(K.cast((1 - y_true) * y_pred, 'float'), axis=0)
    FN = K.sum(K.cast(y_true * (1 - y_pred), 'float'), axis=0)

    precision = TP / (TP + FP + K.epsilon())
    recall = TP / (TP + FN + K.epsilon())

    f1 = 2 * (precision * recall) / (precision + recall + K.epsilon())
    f1 = tf.where(tf.math.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)

## Define Neural Network
Used to see how many parameters are used \
Aim for <1k

In [3]:
model = Sequential()
    
model.add(InputLayer(shape=(images.shape[1], images.shape[2], 1)))
model.add(Conv2D(8, kernel_size=(3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(4, 4))) # max pool helps to keep number of params down

model.add(Flatten())

model.add(Dense(11, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(7, activation='relu')) # can easily add/drop this layer
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))
model.add(Dropout(0.3))

# print out the model structure/summary
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 10, 12, 8)           │              80 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 2, 3, 8)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 48)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 11)                  │             539 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 11)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 7)                   │              84 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 7)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 2)                   │              16 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 2)                   │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 719 (2.81 KB)

 Trainable params: 719 (2.81 KB)

 Non-trainable params: 0 (0.00 B)

## Iterating through different parameters

In [4]:
for param in [0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008, 0.009, 0.01]:
    
    # Step 4: Build the CNN followed by dense neural network
    model = Sequential()
    
    model.add(InputLayer(shape=(images.shape[1], images.shape[2], 1)))
    model.add(Conv2D(8, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPool2D(pool_size=(4, 4))) # max pool helps to keep number of params down
    
    model.add(Flatten())
    
    model.add(Dense(11, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(7, activation='relu')) # can easily add/drop this layer
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))
    model.add(Dropout(0.3))
    
    # print out the model structure/summary
    #model.summary()

    optimizer = keras.optimizers.Adam(learning_rate=0.01)
    #model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[f1_metric]) # use this for f1 as loss
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=["accuracy"]) # use this for accuracy as loss
    model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2, verbose=0)

    # Step 5: Evaluate and plot the confusion matrix
    y_pred_prob = model.predict(X_test)
    y_pred = np.argmax(y_pred_prob, axis=1)
    y_test_labels = np.argmax(y_test, axis=1)

    # Step 7: Calculate and print the F1 score
    f1 = f1_score(y_test_labels, y_pred)
    print(f"Param: {param} | F1 Score: {f1:.2f}")

34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Param: 0.001 | F1 Score: 0.00
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Param: 0.002 | F1 Score: 0.00
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Param: 0.003 | F1 Score: 0.00
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Param: 0.004 | F1 Score: 0.92
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Param: 0.005 | F1 Score: 0.00
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
Param: 0.006 | F1 Score: 0.00
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Param: 0.007 | F1 Score: 0.00
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Param: 0.008 | F1 Score: 0.00
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Param: 0.009 | F1 Score: 0.00
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Param: 0.01 | F1 Score: 0.00


## Print confusion matrix

In [ ]:
y_pred_prob = model.predict(X_test)
y_pred = np.argmax(y_pred_prob, axis=1)
y_test_labels = np.argmax(y_test, axis=1)
cm = confusion_matrix(y_test_labels, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["NEG/CON", "POS"])
disp.plot(cmap=plt.cm.Blues)
plt.title("Confusion Matrix")
plt.show()